In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#hide
! [ -e /content ] && pip install -Uqq fastbook

In [3]:
pip install fastbook --upgrade

In [4]:
import fastbook
fastbook.setup_book()
from fastbook import *

In [5]:
from fastai.vision.widgets import *
from fastai.imports import *

In [6]:
??search_images_ddg

In [ ]:
ims = search_images_ddg('grizzly bear', max_images=150)
len(ims)

In [ ]:
print(ims[0])

In [ ]:
dest = './images/grizzly.jpg'
download_url(ims[0], dest)
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [7]:
bear_types = 'grizzly', 'black', 'teddy'
path = Path('bears')

In [8]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(f'{o} bear', max_images=150)
        download_images(dest, urls=results)

In [9]:
fms = get_image_files(path)
fms

In [10]:
failed = verify_images(fms)
failed

In [11]:
failed.map(Path.unlink);

In [12]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [13]:
dls = bears.dataloaders(path)

In [14]:
dls.valid.show_batch(max_n=4, nrows=1)

In [15]:
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [16]:
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [17]:
#DataAugmentation using RandomResized Crop 
#Displaying different augmented version of 1 example picture
bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [18]:
bears = bears.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [19]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

In [20]:
#Defining Learner for our Model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [21]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [22]:
interp.plot_top_losses(5, nrows=1)

In [23]:
#hide_output
cleaner = ImageClassifierCleaner(learn)
cleaner

In [24]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [25]:
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)


In [26]:
learn.export()

In [27]:
path = Path()
path.ls(file_exts='.pkl')

In [28]:
learn_inf = load_learner(path/'export.pkl')